In [1]:
from PIL import Image
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
import cv2

In [23]:
def hm_to_IMR(heightmap_path, num_levels=5):
    # Read the 16-bit PNG heightmap using OpenCV
    heightmap = cv2.imread(heightmap_path, cv2.IMREAD_UNCHANGED)
    
    # Normalize the values to the range [0, 1]
    heightmap_array = cv2.normalize(heightmap, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    

    # Determine grid size to match heightmap dimensions
    height, width = heightmap_array.shape
    grid_width, grid_height = width // 4, height // 4  # Assuming a 128x128 resolution

    imr_values = [0, 0.25, 0.5, 0.75, 1]

    IMR = np.zeros((grid_height, grid_width))

    # Calculate average brightness in each grid area
    for i in range(grid_height):
        for j in range(grid_width):
            area = heightmap_array[i * 16: (i + 1) * 16, j * 16: (j + 1) * 16]
            average_brightness = np.mean(area)
            
            # Find the two closest values in IMR and assign randomly to one of them
            closest_values = sorted(imr_values, key=lambda x: abs(average_brightness - x))[:2]
            assigned_value = random.choice(closest_values)
            
            # Assign the assigned_value to the IMR grid
            IMR[i][j] = assigned_value
    heightmap = (heightmap.astype('float32') - 32767.5) / 32767.5
    
    return IMR, heightmap

In [24]:
path = './Tiles/Downscaled/tileEflipped_64000_20480.png'
imr, img = hm_to_IMR(path, 5)

In [25]:
IMR_reshaped = imr.reshape(32, 32, 1)

In [27]:
print(img.max())

-0.3180438


## Hex to IMR

In [54]:
def hexmap_to_IMR(hexmap, num_levels=5, height=128, width=128):
    imr_values = [i / (num_levels - 1) for i in range(num_levels)]
    num_hexes = len(hexmap)  # Assuming hexmap is a list of hexes or cells

    IMR = np.zeros(num_hexes)

    for i, hex in enumerate(hexmap):
        # Extract attributes or values from the hex and its neighbors
        attributes = get_hex_attributes(hex, hexmap)

        # Calculate average attribute within the hex and its neighbors
        average_attribute = np.mean(attributes)

        # Find two closest values in IMR and assign randomly to one of them
        closest_values = sorted(imr_values, key=lambda x: abs(average_attribute - x))[:2]
        assigned_value = random.choice(closest_values)

        # Assign the assigned value to the IMR grid representing the hex
        IMR[i] = assigned_value

    # Reshape the IMR to match the height and width of the original heightmap
    IMR = IMR.reshape(height, width)

    return IMR


### Other approach to IMR

In [ ]:

# def HM2IMR(img_path):

#     img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    
#     # Resize the image to 32x32
#     img_resized = cv2.resize(img, (32, 32))
    
#     # Normalize the image to [0, 1]
#     img_normalized = img_resized.astype(np.float32) / 65535.0
    
#     # Define the possible IMR values
#     imr_values = np.array([0.0, 0.25, 0.5, 0.75, 1.0])
    
#     # Initialize an empty array for the IMR
#     imr = np.zeros_like(img_normalized)
    
#     # For each cell in the normalized image...
#     for i in range(img_normalized.shape[0]):
#         for j in range(img_normalized.shape[1]):
#             # Compute the average pixel value within the cell
#             avg_value = np.mean(img_normalized[i, j])
            
#             # Find the closest IMR value to the average pixel value
#             closest_imr_value = imr_values[np.argmin(np.abs(imr_values - avg_value))]
            
#             # Assign the closest IMR value to the cell
#             imr[i, j] = closest_imr_value
    
#     return imr

In [ ]:
# path = './Tiles/Downscaled/tileEflipped_64000_20480.png'
# imr = hm_to_IMR(path)

### Other approach

In [8]:
def HM2IMR(heightmap_path, num_levels=5):
    # Read the 16-bit PNG heightmap using OpenCV
    heightmap = cv2.imread(heightmap_path, cv2.IMREAD_UNCHANGED)

    # Normalize the values to the range [0, 1]
    heightmap_array = cv2.normalize(heightmap, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Determine grid size to match heightmap dimensions
    height, width = heightmap_array.shape
    grid_width, grid_height = width // 4, height // 4

    imr_values = [i / (num_levels - 1) for i in range(num_levels)]

    IMR = np.zeros((grid_height, grid_width))

    # Calculate average brightness in each grid area
    for i in range(grid_height):
        for j in range(grid_width):
            start_i = i * 4
            end_i = min((i + 1) * 4, height)  # Ensure end_i does not exceed image height
            start_j = j * 4
            end_j = min((j + 1) * 4, width)  # Ensure end_j does not exceed image width
            
            area = heightmap_array[start_i:end_i, start_j:end_j]
            
            if area.size > 0:  # Check for empty slices
                average_brightness = np.mean(area)
                
                # Find the two closest values in IMR and assign randomly to one of them
                closest_values = sorted(imr_values, key=lambda x: abs(average_brightness - x))[:2]
                assigned_value = random.choice(closest_values)
                
                # Assign the assigned_value to the IMR grid
                IMR[i][j] = assigned_value

    return IMR, heightmap

In [13]:
path = './Tiles/Downscaled/tileEflipped_64000_20480.png'
imr, img = HM2IMR(path, 5)

In [14]:
print(imr.shape)
print(imr)

(32, 32)
[[0.5  0.5  0.5  ... 0.5  0.5  0.25]
 [0.25 0.5  0.25 ... 0.25 0.25 0.25]
 [0.5  0.5  0.25 ... 0.5  0.5  0.25]
 ...
 [0.   0.   0.25 ... 0.25 0.25 0.  ]
 [0.25 0.   0.25 ... 0.   0.25 0.  ]
 [0.25 0.25 0.25 ... 0.25 0.25 0.  ]]


## Testing

In [3]:
import os
import numpy as np
import cv2
import random

In [14]:
def cust_data_generator(dir_path, batch_size):
    # Get the list of all files in the directory
    file_list = os.listdir(dir_path)
    num_files = len(file_list)

    while True:
        # shuffle
        np.random.shuffle(file_list)

        for i in range(0, num_files, batch_size):
            # get a list of filenames of a batch
            batch_files = file_list[i : i + batch_size]

            batch_images = []
            batch_IMRS = []

            for file in batch_files:
                file_path = os.path.join(dir_path, file)

                # read the image and generate IMR
                img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                arr = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                height, width = arr.shape
                # Assuming a resolution of 128 x 128
                grid_width, grid_height = width // 4, height // 4

                imr_values = [0, 0.25, 0.5, 0.75, 1]

                IMR = np.zeros((grid_height, grid_width))

                for i in range(grid_height):
                    for j in range(grid_width):
                        start_i = i * 4
                        end_i = min((i + 1) * 4, height)
                        start_j = j * 4
                        end_j = min((j + 1) * 4, width)

                        area = arr[start_i:end_i, start_j:end_j]

                        if area.size > 0:
                            avg_brightness = np.mean(area)

                            closest_values = sorted(imr_values, key=lambda x: abs(avg_brightness - x))[:2]
                            assigned_value = random.choice(closest_values)
                            IMR[i][j] = assigned_value
                
                IMR = IMR.reshape(32, 32, 1)
                img = img.
                batch_images.append(img)
                batch_IMRS.append(IMR)
                print(IMR.shape)
                print(img.shape)
                

In [ ]:
cust_data_generator('./Tiles/Train', 64)

In [2]:
def sketch2IMR(dir_path):
    
    # read the image and generate IMR
    img = cv2.imread(dir_path, cv2.IMREAD_GRAYSCALE)
    arr = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    height, width = arr.shape
    # Assuming a resolution of 128 x 128
    grid_width, grid_height = width // 4, height // 4

    imr_values = [0, 0.25, 0.5, 0.75, 1]

    IMR = np.zeros((grid_height, grid_width))

    for i in range(grid_height):
        for j in range(grid_width):
            start_i = i * 4
            end_i = min((i + 1) * 4, height)
            start_j = j * 4
            end_j = min((j + 1) * 4, width)
            area = arr[start_i:end_i, start_j:end_j]

            if area.size > 0:
                avg_brightness = np.mean(area)
                closest_values = sorted(imr_values, key=lambda x: abs(avg_brightness - x))[:2]
                assigned_value = random.choice(closest_values)
                IMR[i][j] = assigned_value
                
    IMR_reshaped = IMR.reshape(32, 32, 1)
        
    return IMR_reshaped